In [181]:
import pandas as pd
import glob
import numpy as np

In [103]:
cd /home/jovyan/work/02.ResDev/05.MC38_TAA/

/home/jovyan/work/02.ResDev/05.MC38_TAA


### step1: Get sample sample of cancer,paired and normaltissue ###

In [364]:
MC38_sample_info=pd.read_csv("/home/jovyan/work/01.dataset/11.MC38_Xomic/MC38_data_info.txt",header=0,sep='\t')
Paired_sample_info=pd.read_csv("/home/jovyan/work/01.dataset/12.C57BL6_wildtype_RNAseq/C57BL6_normal_tissue_info.txt",header=0,sep="\t")
Normal_tissue_info=pd.read_csv("/home/jovyan/work/01.dataset/12.C57BL6_wildtype_RNAseq/01.atlas_normal_tissue/sample.list",names=("sample","path"),sep="\t")

In [354]:
MC38_sample_info_ctr=MC38_sample_info[MC38_sample_info.group=="control"]
MC38_sample = MC38_sample_info_ctr["run(SRA)"].to_list()
Paired_sample = Paired_sample_info["run(SRA)"].to_list()
Normal_tissue_sample =Normal_tissue_info["sample"].to_list()

In [355]:
print(MC38_sample)
print(Paired_sample)
print(Normal_tissue_sample)

['SRR19868734', 'SRR19868735', 'SRR19751383', 'SRR19751384', 'SRR19751385', 'SRR18925573', 'SRR18925574', 'SRR18925575', 'SRR18925576', 'SRR18925577', 'SRR18925578', 'SRR18925579', 'SRR18925580', 'SRR18925581', 'SRR18335103', 'SRR18335102', 'SRR18335110', 'SRR18335109', 'SRR18335101']
['SRR17901548', 'SRR17901549', 'SRR17901550', 'SRR17901551', 'SRR17901552', 'SRR17901553', 'SRR17901554', 'SRR17901555', 'SRR17901560', 'SRR17901561', 'SRR17901562', 'SRR17901563', 'SRR14807913', 'SRR14807914', 'SRR14807915']
['ERR2130614', 'ERR2130615', 'ERR2130616', 'ERR2130617', 'ERR2130618', 'ERR2130619', 'ERR2130620', 'ERR2130621', 'ERR2130622', 'ERR2130623', 'ERR2130624', 'ERR2130625', 'ERR2130626', 'ERR2130627', 'ERR2130628', 'ERR2130629', 'ERR2130630', 'ERR2130631', 'ERR2130632', 'ERR2130633', 'ERR2130634', 'ERR2130635', 'ERR2130636', 'ERR2130637', 'ERR2130638', 'ERR2130639', 'ERR2130640', 'ERR2130641', 'ERR2130642', 'ERR2130643', 'ERR2130644', 'ERR2130645', 'ERR2130646', 'ERR2130647', 'ERR2130648

### step2: Merge all quant.genes.sf files generated by salmon and Transform format of df_tpm_all and df_readnum_all to the one alike Xena_UCSC¶###

In [356]:
## give all the paths contain salmon quantify results
all_files=glob.glob("/home/jovyan/work/02.ResDev/05.MC38_TAA/SRR*/03.Qu*/SRR*quant/quant.genes.sf")
all_files2=glob.glob("/home/jovyan/work/02.ResDev/06.C57BL6_wildtype/ERR*/03.Quant/ERR*_salmon_quant/quant.genes.sf")

series_cancervspaired_tpm_all=[]
series_cancervspaired_readNum_all=[]
series_cancervsnormal_tpm_all=[]

## collect all need samples in path1
for file in all_files:
    '''
    # because samplename in matrix files generated from Xena_UCSC have origin prefix, such as 'TCGA' or 'GTEX'
    # DEGselect pipeline judge the sample as tumor or paired by distinguishing the origin prefix,
    # So we must add the prefix to the samplename
    '''
    sampleName=file.split("/")[6]
    #print(sampleName)
    if sampleName in MC38_sample:
        sampleName="TCGA-"+file.split("/")[6]
    elif sampleName in Paired_sample:
        sampleName="GTEX-"+file.split("/")[6]
    elif sampleName in Normal_tissue_sample:
        sampleName="GTEX-"+file.split("/")[6]
    else:
        continue
    #print(sampleName)
    df=pd.read_csv(file,sep='\t',header=0, index_col="Name")
    df_tpm, df_rn = df["TPM"], df["NumReads"]
    df_tpm.name, df_rn.name = sampleName,sampleName
    series_cancervspaired_tpm_all.append(df_tpm)
    series_cancervspaired_readNum_all.append(df_rn)
    series_cancervsnormal_tpm_all.append(df_tpm)
df_cancervspaired_tpm = pd.concat(series_cancervspaired_tpm_all,axis=1)
df_cancervspaired_readNum = pd.concat(series_cancervspaired_readNum_all,axis=1)

## collect all need samples in path2
for file in all_files2:
    '''
    # because samplename in matrix files generated from Xena_UCSC have origin prefix, such as 'TCGA' or 'GTEX'
    # DEGselect pipeline judge the sample as tumor or paired by distinguishing the origin prefix,
    # So we must add the prefix to the samplename
    '''
    sampleName=file.split("/")[6]
    #print(sampleName)
    if sampleName in MC38_sample:
        sampleName="TCGA-"+file.split("/")[6]
    elif sampleName in Paired_sample:
        sampleName="GTEX-"+file.split("/")[6]
    elif sampleName in Normal_tissue_sample:
        sampleName="GTEX-"+file.split("/")[6]
    else:
        continue
    df=pd.read_csv(file,sep='\t',header=0, index_col="Name")
    df_tpm, df_rn = df["TPM"], df["NumReads"]
    df_tpm.name, df_rn.name = sampleName,sampleName
    series_cancervsnormal_tpm_all.append(df_tpm)
df_cancervsnormal_tpm=pd.concat(series_cancervsnormal_tpm_all,axis=1)


In [357]:
## Transform matrix data to Logarithmic format
'''
# TPM and counts data from Xena_UCSC was logorithmic
# To fit with the pipeline, we all transform the data format
#
'''
df_cancervspaired_tpm = np.log2(df_cancervspaired_tpm+0.001)
df_cancervspaired_readNum = np.log2(df_cancervspaired_readNum+1)
df_cancervsnormal_tpm = np.log2(df_cancervsnormal_tpm+0.001)

In [358]:
## filter all matrix and df_cancervspaired_readNum-specific transforming of index name
'''
# Filter rows according genename not in the 'ensembleID2genename2biotype.txt'
# and Replace the genename in the 'df_cancervspaired_readNum' with the one in 'ensembleID2genename2biotype.txt'
'''
df_ensemid2genename=pd.read_csv("/home/jovyan/work/00.database/16.annot_mouse/ensembleID2genename2biotype.txt",sep="\t",header=0,index_col=0)
geneid_list = df_ensemid2genename.index.to_list()
#print(geneid_list)
print(len(df_cancervspaired_tpm))
print(len(df_cancervsnormal_tpm))
print(len(df_cancervspaired_readNum))

df_cancervspaired_tpm2 = df_cancervspaired_tpm.loc[geneid_list]
df_cancervsnormal_tpm2 = df_cancervsnormal_tpm.loc[geneid_list]
df_cancervspaired_readNum2 = df_cancervspaired_readNum.merge(df_ensemid2genename,left_index=True, right_index=True,how="inner")
df_cancervspaired_readNum2 = df_cancervspaired_readNum2.set_index("gene_name")
df_cancervspaired_readNum2 = df_cancervspaired_readNum2.drop(columns=["gene_biotype"])

print(len(df_cancervspaired_tpm2))
print(len(df_cancervsnormal_tpm2))
print(len(df_cancervspaired_readNum2))

## remove the whitespace in the dataframe rownames
df_cancervspaired_tpm2.index = df_cancervspaired_tpm2.index.str.strip()
df_cancervsnormal_tpm2.index = df_cancervsnormal_tpm2.index.str.strip()
df_cancervspaired_readNum2.index = df_cancervspaired_readNum2.index.str.strip()

56748
56748
56748
53096
53096
53096


In [411]:
## cluster sampleid of file of "df_cancervspaired_tpm2", otherwise heatmap of TMP will cannot cluster 
cols = list(df_cancervspaired_tpm2.columns)
cancer_sample=[]
paired_sample=[]
for sampleid in cols:
    if "TCGA" in sampleid:
        cancer_sample.append(sampleid)
    if "GTEX" in sampleid:
        paired_sample.append(sampleid)
samplelist = cancer_sample+paired_sample
df_cancervspaired_tpm2 = df_cancervspaired_tpm2[samplelist]

In [412]:
## saving all collected sample's dataframe file
df_cancervspaired_tpm2.to_csv("/home/jovyan/work/docker_data/17.jupyter_script/MC38_vs_paired_TPM.txt",sep='\t')
df_cancervspaired_readNum2.to_csv("/home/jovyan/work/docker_data/17.jupyter_script/MC38_vs_paired_count.txt",sep='\t')
df_cancervsnormal_tpm2.to_csv("/home/jovyan/work/docker_data/17.jupyter_script/MC38_vs_Tissue_TPM.txt",sep='\t')

## Appendix: make phenotype.txt file ##

In [390]:
atlas_info = pd.read_csv("/home/jovyan/work/01.dataset/12.C57BL6_wildtype_RNAseq/01.atlas_normal_tissue/a_assay_Simon.txt",header=0,sep="\t")
atlas_info = atlas_info[["Assay Name","Factor Value[organism part]"]]
atlas_info = atlas_info.rename(columns={"Factor Value[organism part]":"_primary_site","Assay Name":"sample"})
atlas_info["_study"] = "GTEX"
atlas_info["_sample_type"] = "Tissue Normal"
atlas_info["sample"] = atlas_info["_study"]+"-"+atlas_info["sample"]
atlas_info["_primary_site"] = [tissue.capitalize() for tissue in atlas_info._primary_site.to_list()]
atlas_info.to_csv("/home/jovyan/work/docker_data/17.jupyter_script/MC38Tissue_phenotype.txt",sep='\t',index=False)

## testing region

In [203]:
all_files2=glob.glob("/home/jovyan/work/02.ResDev/06.C57BL6_wildtype/ERR*/03.Quant/ERR*_salmon_quant/quant.genes.sf")

In [289]:
import pandas as pd

#initialize a dataframe
df = pd.DataFrame(
	[[21, 'Amol', 72, 67],
	[23, 'Lini', 78, 69],
	[32, 'Kiku', 74, 56],
	[52, 'Ajit', 54, 76]],
	columns=['rollno', 'name', 'physics', 'botony'])

print('DataFrame with default index\n', df)

#set column as index
df = df.set_index('rollno')

print('\nDataFrame with column as index\n',df)

DataFrame with default index
    rollno  name  physics  botony
0      21  Amol       72      67
1      23  Lini       78      69
2      32  Kiku       74      56
3      52  Ajit       54      76

DataFrame with column as index
         name  physics  botony
rollno                       
21      Amol       72      67
23      Lini       78      69
32      Kiku       74      56
52      Ajit       54      76


### step2: Transform format of df_tpm_all and  df_readnum_all to the one alike Xena_UCSC

In [241]:
df1 = data.frame(x=1:3,y=1:3,row.names=c('r1','r2','r3'))
df2 = data.frame(z=5:7,row.names=c('r5','r6','r7'))

SyntaxError: invalid syntax (695919881.py, line 1)

In [180]:
data= np.log10(data)
data

,Salary,years
0,2.00000,4.0
1,3.00000,6.0
2,4.00000,7.0
3,5.60206,7.0


In [153]:
for letter in "Jessica":
    if letter == "i":
        break
    print(letter)

J
e
s
s


In [405]:
# a list of numbers
my_numbers = [10, 8, 3, 22, 33, 7, 11, 100, 54]

#sort list in-place in ascending order
my_numbers.sort()

#print modified list
print(my_numbers)

#output

#[3, 7, 8, 10, 11, 22, 33, 54, 100]

[3, 7, 8, 10, 11, 22, 33, 54, 100]
